Drumplot (Helicorder) With Data Select and Filtering
--

**What and How**

The purpose of this notebook is to provide non-seismologists with a relatively easy to use tool to make drumplots (helicorders) of a day of seismic data. In particular, it provides a capability to filter the data before plotting. This was identified as a short coming of existing products at the time of the April 2016 White Island eruption. Filtering can be important to enhance very long period (VLP) signals in seismic waveforms that can accompany explosive eruptions, but it can also enhance other signals with a specific frequency content.

Data are acquired from GeoNet's FDSN web servers. The archive server is used as the first choice, and if that does not contain the data (it only keeps data older than 7 days) then the near real-time server will be used instead (which contains the last 8 days of data.

Large signals can produce a trace that extends over much of the drumplot and obscures other parts of the plot. For that reason, large amplitude waves are clipped before plotting.


**Instructions**
1. Run the notebook. To do this, click 'Cell' in the dropdown menu, and then click 'Run All'.
2. Select the date and the data stream, and then click 'Retrieve data'.
3. This may take a few moments. Once the data have been retrieved, a message will be displayed.
4. If you want to filter the data, select the box labelled Filter? The default is no filtering. If you want to filter the waveform, you also need to select the bandpass filter range. The default is 0.1 to 10 Hz.
5. Depending on the amplitude of the signal, it may be necessary to adjust the plot gain, this scales the traces up or down. Some experimentation may be needed to find the best value. Try the default value first, it is suitable for many drumplots.
6. Click 'Show on screen' to display the drumplot below.
7. Large amplitudes are automatically clipped. The clip value is currently not adjustable by a user.
8. You can repeat steps 4-6 as many times as you want, adjusting filter parameters and plot gains, to get the drumplot looking the way you want. Each time you click on 'Show on screen', a new drumplot will be produced and will show below any existing drumplots.
9. To produce an output file suitable for printing, click on 'Output to file'. This will produce a file that is visible in the Jupyter Hub file manager tab in your browser. To save that file, click on the file name to open a view of the helicorder in another browser tab, then right click on the image and select 'Save image as ...'.
10. The image will be named station.location.component.network_yyyymmdd_filter.drum.png. Examples are WIZ.10.HHZ.NZ_20171031_bandpass_0.1-4.6.drum.png and WIZ.10.HHZ.NZ_20171031_nofilt.drum.png.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from IPython.display import HTML
from obspy.core import UTCDateTime, Stream
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, Layout, HBox, VBox

%matplotlib inline

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#GeoNet's FDSN web servers
arc_client = 'http://service.geonet.org.nz'
nrt_client = 'http://beta-service-nrt.geonet.org.nz'

In [ ]:
current = False

In [ ]:
#plot initialisation
nofilt = False
bpfilt = False
lpfilt = False
hpfilt = False
corners = 2
zerophase = False
defscale = 1e4
# defscale_acou = 1e2
scale = defscale
magn = 1

#the next default filter values are set in widget functions. but if the default frequencies are not altered
#they are not set
global blp, bhp, llp, hhp #don't like this but not sure better solution as need visible in a function
blp = 2
bhp = 6
llp = 1
hhp = 5

#size ratio to match aspect ratio of print paper
xsize = 3200
ysize = int(xsize/1.4) 
size = (xsize,ysize)

#large trace clip
clip = 100000

#possible streams to select
streams = ['WIZ.30.HDF.NZ', 'WIZ.31.HDF.NZ', 'TMVZ.10.HHZ.NZ', 'TMVZ.10.HHN.NZ', 'TMVZ.10.HHE.NZ', 'NMEZ.10.EHZ.NZ', 'NBEZ.10.EHZ.NZ', 'NEZ.11.EHZ.NZ', 
        'KHEZ.10.HHZ.NZ', 'ETVZ.10.HHZ.NZ', 'OTVZ.10.HHZ.NZ', 'WIZ.10.HHZ.NZ', 'WSRZ.10.HHZ.NZ', 'MAVZ.10.HHZ.NZ', 'FWVZ.10.HHZ.NZ', 'WHVZ.10.HHZ.NZ', 
        'TRVZ.10.HHZ.NZ', 'GLKZ.10.HHZ.NZ', 'KRVZ.10.EHZ.NZ']
#sort and add hint as first element
streams.sort()
streams.insert(0, 'select stream')

In [ ]:
def selsite(s):
  global site, stn, loc, cmp, net
  site = s['new']
  
  stn = site.split(".")[0]
  loc = site.split(".")[1]
  cmp = site.split(".")[2]
  net = site.split(".")[3]

def seldate(d):
  global pdate
  date = d['new']
  pdate = date.strftime("%Y%m%d")
  
def retrieve(r):
    global tr
    time = UTCDateTime(pdate)
    try:
        client = Client(arc_client)
        starc = client.get_waveforms(str(net), str(stn), str(loc), str(cmp), time, time + 86400, attach_response=True)
        print('arc client successful')
    except:
        print('arc client not successful')
        starc = Stream()
    try:
        client = Client(nrt_client)
        stnrt = client.get_waveforms(str(net), str(stn), str(loc), str(cmp), time, time + 86400, attach_response=True)
        print ('nrt client successful')
    except:
        print('nrt client not successful')
        stnrt = Stream()
  
    st = starc + stnrt
    st.merge(fill_value = 0.0) #merge streams, fill gaps with zeros
    st.remove_sensitivity()
    
    tr = st[0]
    tr.data *= 1e9 #convert from m/s to nm/s
    tr.data = np.clip(tr.data, clip*-1, clip)
    print ('data retrieved')
    print (tr)

#no filter
def nofilter(a):
    #no filter check box
    global nofilt  
    nofilt = a['new']
    
#bandpass
def bpfilter(f):
    #bandpass filter check box
    global bpfilt
    bpfilt = f['new']
        
def bandpass(b):
    #two slider values
    global blp, bhp
    blp, bhp  = b['new']
        
#lowpass
def lpfilter(d):
    #lowpass filter check box
    global lpfilt  
    lpfilt = llp
    lpfilt = d['new']
#     lpfilt = d
    
def lowpass(e):
    #single slider value
    global llp
    llp = e['new']
    
#highpass
def hpfilter(h):
    #highpass filter check box
    global hpfilt  
    hpfilt = h['new']
    
def highpass(i):
    #single slider value
    global hhp
    hhp = i['new']
#     hhp = i

#plot gain
def gain(g):
  global magn, scale
  magn = g['new']
  scale = defscale / magn

def screen(c):
    if (nofilt):
        title = site + '_' + pdate + '_nofilt'+'_plot-gain_'+str(magn)
        tr.plot(type='dayplot', linewidth = 0.3, vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title)
    if (bpfilt):
        trcp = tr.copy()
        trcp.filter('bandpass', freqmin=blp, freqmax=bhp, corners=corners, zerophase=zerophase)
        title = site+'_'+pdate+'_bandpass_'+str(blp)+'-'+str(bhp)+'_plot-gain_'+str(magn)
        trcp.plot(type='dayplot', linewidth = 0.3, vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title)
    if (lpfilt):
        trcp = tr.copy()
        trcp.filter('lowpass', freq=llp, corners=corners, zerophase=zerophase)
        title = site+'_'+pdate+'_lowpass_'+str(llp)+'_plot-gain_'+str(magn)
        trcp.plot(type='dayplot', linewidth = 0.3, vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title)
    if (hpfilt):
        trcp = tr.copy()
        trcp.filter('highpass', freq=hhp, corners=corners, zerophase=zerophase)
        title = site+'_'+pdate+'_highpass_'+str(hhp)+'_plot-gain_'+str(magn)
        trcp.plot(type='dayplot', linewidth = 0.3, vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title)
    

def output(o):
    if (nofilt):
        title = site+'_'+pdate+'_nofilt'+'_plot-gain_'+str(magn)
        tr.plot(type='dayplot', linewidth = 0.3, vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title, outfile=title+'.drum.png')
    if (bpfilt):
        trcp = tr.copy()
        trcp.filter('bandpass', freqmin=blp, freqmax=bhp, corners=corners, zerophase=zerophase)
        title = site+'_'+pdate+'_bandpass_'+str(blp)+'-'+str(bhp)+'_plot-gain_'+str(magn)
        trcp.plot(type='dayplot', linewidth = 0.3, vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title, outfile=title+'.drum.png')
    if (lpfilt):
        trcp = tr.copy()
        trcp.filter('lowpass', freq=llp, corners=corners, zerophase=zerophase)
        title = site+'_'+pdate+'_lowpass_'+str(llp)+'_plot-gain_'+str(magn)
        trcp.plot(type='dayplot', linewidth = 0.3, vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title, outfile=title+'.drum.png')
    if (hpfilt):
        trcp = tr.copy()
        trcp.filter('highpass', freq=hhp, corners=corners, zerophase=zerophase)
        title = site+'_'+pdate+'_highpass_'+str(hhp)+'_plot-gain_'+str(magn)
        trcp.plot(type='dayplot', linewidth = 0.3, vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title, outfile=title+'.drum.png')
    

In [ ]:
s = widgets.Select(
#     options=['select stream', 'WIZ.30.HDF.NZ', 'WIZ.31.HDF.NZ', 'TMVZ.10.HHZ.NZ', 'TMVZ.10.HHN.NZ', 'TMVZ.10.HHE.NZ', 'NMEZ.10.EHZ.NZ', 'NBEZ.10.EHZ.NZ', 'NEZ.11.EHZ.NZ', 'KHEZ.10.HHZ.NZ', 'ETVZ.10.HHZ.NZ', 'OTVZ.10.HHZ.NZ', 'WIZ.10.HHZ.NZ', 'WSRZ.10.HHZ.NZ', 'MAVZ.10.HHZ.NZ', 'FWVZ.10.HHZ.NZ', 'WHVZ.10.HHZ.NZ', 'TRVZ.10.HHZ.NZ', 'GLKZ.10.HHZ.NZ', 'KRVZ.10.EHZ.NZ'],
    options=streams,

    value='select stream',
    description='Stream:',
    disabled=False
)
s.observe(selsite, names='value')

d = widgets.DatePicker(
  description='Date:')
d.observe(seldate, names='value')

r=widgets.Button(
    description='Retrieve data',
    disabled=False,
    button_style='success',
    tooltip='Retrieve data for plotting')
r.on_click(retrieve)

widgets.HBox([d, s, r])

In [ ]:
#no filter check box
a=widgets.Checkbox(
    description='NF?',
    value=False,
    tooltip='No filtering',
    disabled=False,
    layout=widgets.Layout(width='10%'))
a.observe(nofilter,names='value')

#bandpass check box
f=widgets.Checkbox(
    description='BP?',
    value=False,
    tooltip='Set filtering',
    disabled=False,
    layout=widgets.Layout(width='10%'))
f.observe(bpfilter,names='value')

#bandpass slider
b=widgets.FloatRangeSlider(
    value=[2, 6],
    min=0.5,
    max=10,
    step=0.5,
    description='band-pass:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    slider_color='white')
b.observe(bandpass,names='value')

#lowpass filter check box
d=widgets.Checkbox(
    description='LP?',
    value=False,
    tooltip='Set filtering',
    disabled=False,
    layout=widgets.Layout(width='10%'))
d.observe(lpfilter,names='value')

#lowpass slider
e=widgets.FloatSlider(
    value=1,
    min=0.01,
    max=2,
    step=0.01,
    description='low-pass:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    slider_color='white')
e.observe(lowpass, names='value')

#highpass filter check box
h=widgets.Checkbox(
    description='HP?',
    value=False,
    tooltip='Set filtering',
    disabled=False,
    layout=widgets.Layout(width='10%'))
h.observe(hpfilter,names='value')

#highpass slider
i=widgets.FloatSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='high-pass:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    slider_color='white')
# interact(highpass, i)
i.observe(highpass, names='value')

#gain slider
g=widgets.FloatSlider(
    value=1,
    min=0.1,
    max=50,
    step=0.1,
    description='plot gain:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    slider_color='white')
g.observe(gain, names='value')

c=widgets.Button(
    description='Show on screen',
    disabled=False,
    button_style='success',
    tooltip='Make drumplot')
c.on_click(screen)

o=widgets.Button(
    description='Output to file',
    disabled=False,
    button_style='success',
    tooltip='Make file copy')
o.on_click(output)

box0 = HBox([a,g]) #no filter
box1 = HBox([f,b,g]) #bandpass
box2 = HBox([d,e,g]) #lowpass
box3 = HBox([h,i,g]) #highpass
box_view = widgets.HBox([c,o])
VBox([box0, box1, box2, box3, box_view])
